In [1]:
from functions import objective
from hyperopt.pyll import scope
from hyperopt import Trials, fmin, hp, tpe
from loguru import logger

c:\Users\pikob\Documents\GitHub\portfolio-MADS-ML-ElinevDijk\.venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("my_experiment")

with mlflow.start_run():
    mlflow.log_param("param1", 10)
    mlflow.log_metric("accuracy", 0.85)


2025/09/20 19:58:41 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/20 19:58:41 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [3]:
import mlflow

# Set tracking URI to local SQLite DB in current folder
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set experiment name
mlflow.set_experiment("test_experiment")

with mlflow.start_run():
    mlflow.log_param("param1", 123)
    mlflow.log_metric("accuracy", 0.9)

print("Run logged.")


2025/09/20 19:37:17 INFO mlflow.tracking.fluent: Experiment with name 'test_experiment' does not exist. Creating a new experiment.


Run logged.


In [4]:
import mlflow

# Set up tracking to a local SQLite file
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Set or create an experiment
mlflow.set_experiment("test_experiment")

# Start and log a run
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("accuracy", 0.89)

# Retrieve experiment and list runs
experiment = mlflow.get_experiment_by_name("test_experiment")
if experiment is None:
    print("No experiment found!")
else:
    print(f"Experiment ID: {experiment.experiment_id}")
    runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
    print(f"Number of runs found: {len(runs)}")
    if len(runs) > 0:
        print(runs[['run_id', 'status', 'start_time']])


Experiment ID: 2
Number of runs found: 2
                             run_id    status                       start_time
0  5180ca43dcaa41ffbe3c9d2660fc2f27  FINISHED 2025-09-20 18:03:32.765000+00:00
1  02cff6287cf141639eb002bd9f3b8db9    FAILED 2025-09-20 18:03:06.944000+00:00


In [3]:
setup_mlflow("mlflow_database")

search_space = {
    "filters": scope.int(hp.quniform("filters", 16, 128, 8)),
    "units1": scope.int(hp.quniform("units1", 32, 128, 8)),
    "units2": scope.int(hp.quniform("units2", 32, 128, 8)),
    "use_batchnorm": hp.choice("use_batchnorm", [True, False]),
    "use_maxpool": hp.choice("use_maxpool", [True, False]),
    "dropout_conv_rate": hp.uniform("dropout_conve_rate", 0.0, 0.5),
    "dropout_dense_rate": hp.uniform("dropout_dense_rate", 0.0, 0.5),
}

best_result = fmin(
    fn=objective, space=search_space, algo=tpe.suggest, max_evals=2, trials=Trials()
)

logger.info(f"Best result: {best_result}")

2025/09/20 20:18:39 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/20 20:18:39 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


  0%|          | 0/2 [00:00<?, ?trial/s, best loss=?]

2025-09-20 20:18:39.352 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\pikob\.cache\mads_datasets\fashionmnist
2025-09-20 20:18:39.352 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\pikob\.cache\mads_datasets\fashionmnist\fashionmnist.pt
2025-09-20 20:18:39.386 | INFO     | functions:get_device:37 - Using cpu
2025-09-20 20:18:39.686 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellog\20250920-201839
2025-09-20 20:18:40.923 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:29<00:00,  3.35it/s]
2025-09-20 20:19:20.503 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.9991 test 1.7191 metric ['0.2875']
100%|##########| 100/100 [00:31<00:00,  3.15it/s]
2025-09-20 20:20:02.767 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.7292 test 1.6730 metric ['0.2908']
100%|########

 50%|█████     | 1/2 [01:23<01:23, 83.43s/trial, best loss: 1.6730441355705261]

2025-09-20 20:20:02.802 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\pikob\.cache\mads_datasets\fashionmnist
2025-09-20 20:20:02.803 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\pikob\.cache\mads_datasets\fashionmnist\fashionmnist.pt
2025-09-20 20:20:02.828 | INFO     | functions:get_device:37 - Using cpu
2025-09-20 20:20:02.937 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellog\20250920-202002
2025-09-20 20:20:02.939 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:08<00:00, 11.42it/s]
2025-09-20 20:20:15.309 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.7859 test 1.1784 metric ['0.5387']
100%|##########| 100/100 [00:08<00:00, 11.41it/s]
2025-09-20 20:20:27.626 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.1887 test 0.8554 metric ['0.6667']
100%|########

100%|██████████| 2/2 [01:48<00:00, 54.15s/trial, best loss: 0.8554385775327682]

2025-09-20 20:20:27.656 | INFO     | __main__:<module>:17 - Best result: {'dropout_conve_rate': np.float64(0.13867199653017792), 'dropout_dense_rate': np.float64(0.14469338130759574), 'filters': np.float64(112.0), 'units1': np.float64(112.0), 'units2': np.float64(72.0), 'use_batchnorm': np.int64(1), 'use_maxpool': np.int64(0)}
